# Task II - Estimating Service Metrics from Device Statistics

The objective of this task is to estimate the frame rate of a VoD service from the device statistics of a VoD server. Our approach is to gather observations from the system and apply linear regression on this data to estimate the service metric Y from device statistics X. For this task, we use the trace with 3600 observations described above.

Your task is to compute (i.e., to train) a linear model M that accurately maps device statistics onto service metrics.

You train and test your model M with the so-called validation-set technique. This technique entails that you split the set of observations into two parts: the training set for computing the model M and the test set for evaluating the accuracy of M. From the complete set of observations, you select uniformly at random 70% of the observations (i.e., 2520 observations) to form the training set and then assign the remaining 30% i.e., 1080 observations) to the test set.

## Evaluate the Accuracy of Service Metric Estimation

### (a) Model Training - use linear regression to train a model M with the training set. Provide the coefficients (Θ0, ..., Θ9) of your model M. (Θ0 is the offset.)

In [18]:
import pandas
import numpy
import matplotlib
import pylab
import sklearn.model_selection
import sklearn.linear_model

In [6]:
# import dataset_x and dataset_y
data_set_x = pandas.read_csv('X.csv')
data_set_y = pandas.read_csv('Y.csv')

# join operations similar to relational databases on field 'TimeStamp'
data_set = pandas.merge(data_set_x, data_set_y, on='TimeStamp')

In [20]:
(data_set_train, data_set_test) = sklearn.model_selection.train_test_split(data_set, test_size=0.3)

In [44]:
linear_regressor = sklearn.linear_model.LinearRegression()
X = data_set_train.iloc[:, :-1] # all lines, all columns except the last
Y = data_set_train.iloc[:, -1] # all lines and only last column
linear_regressor.fit(X, Y)
print('The coefficients of our model M is', linear_regressor.coef_)

The coefficients of our model M is [-1.62269693e-03 -9.41774587e-02  8.35119258e-02 -3.90896625e-03
 -9.31772953e-05 -1.14407285e-03  1.89527270e-05 -8.81928547e-02
 -4.01769720e-02 -2.20313924e-05]


###  (b) Accuracy of Model M - compute the estimation error of M over the test set. We define the estimation error as the Normalized Mean Absolute Error (NMAE). As a baseline for M, use a na¨ıve method which relies on Y values only. For each x ∈ X it predicts a constant value ¯y  hich is the sample mean of the samples yi in the training set. Compute y for the na¨ıve method for the training set and compute the NMAE for the test set.